In [766]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [767]:
survey = pd.read_csv('./survey_or.csv')
interview = pd.read_csv('./Stakeholders_FRs_Bens.csv')
it_stakeholders = pd.read_csv('./DeIDStakeholderLog_v1.csv',index_col=0)

Preprocess Interview Data

?? G-STK-001

STK-011 twice for different notebooks

In [769]:
index_list = interview['Stakeholder ID'].tolist()

index_list[0] = 'STK-001'

it_stakeholders = it_stakeholders[it_stakeholders['Stakeholder ID'].isin(index_list)]
it_stakeholders = it_stakeholders.set_index('Stakeholder ID')

renameit = {'Primary stakeholder group (from mapping)':'Stakeholder Group'}
it_stakeholders.rename(columns = renameit, inplace = True)
it_stakeholders.fillna('Unknown', inplace=True)


Preprocess Survey Data

In [771]:
start = survey.columns.get_loc('Q2.1')
end = survey.columns.get_loc('Q2.24')
sv_stakeholders = survey.iloc[1:,start:end+1]
sv_stakeholders = sv_stakeholders.drop('Q2.3',axis=1)

#FBMH
sv_stakeholders['Division_FBWH'] = sv_stakeholders.apply(
    lambda row: ''.join([
        str(x) for x in [row['Q2.6'], row['Q2.7'], row['Q2.8'], row['Q2.9']] if pd.notna(x)]),axis=1)

#FSE
sv_stakeholders['Division_FSE'] = sv_stakeholders.apply(
    lambda row: ''.join([
        str(x) for x in [row['Q2.18'], row['Q2.19'], row['Q2.20']] if pd.notna(x)]),axis=1)

#All Division 
sv_stakeholders['Department or Division'] = sv_stakeholders.apply(
    lambda row: ''.join([
        str(x) for x in [row['Q2.12'], row['Q2.15'], row['Division_FBWH'],row['Division_FSE']] if pd.notna(x)]),axis=1)

#All School
sv_stakeholders['School'] = sv_stakeholders.apply(
    lambda row: ''.join([
        str(x) for x in [row['Q2.5'], row['Q2.11'], row['Q2.14'],row['Q2.17']] if pd.notna(x)]),axis=1)

#All Institute/Building/Affilitation
sv_stakeholders['Institute/Building/Affilitation'] = sv_stakeholders.apply(
    lambda row: ''.join([
        str(x) for x in [row['Q2.22'], row['Q2.24']] if pd.notna(x)]),axis=1)

sv_stakeholders = sv_stakeholders[['Q2.1', 'Q2.2','School','Department or Division','Institute/Building/Affilitation']]

rename = {'Q2.1':'Stakeholder Group', 'Q2.2':'Faculty'}
sv_stakeholders.rename(columns = rename, inplace = True)
sv_stakeholders.fillna('Unknown', inplace=True)

? the school HUMS and PS do not have questions to know the division

? if Q2.14 for Ps is school and Q2.15 for ps is division 

？match Q2.12, Q2.20, Q2.9 with division in interview
'Technical Services':'Technical'

? for stakeholder group match 
'Research supporting (Other)'= 'Academic Research Support',
Technical Support= Research supporting (Technical)
None of the above = NaN

Matching 

In [774]:
replace_sg = {'Postgraduate researcher (PGR)':'PGRs','Early-career researcher':'Early Career Researchers',
             'Research supporting (Other)':'Academic Research Support','Research supporting (Technical)':'Technical Support',
             'None of the above':'Unknow'}

replace_fc = {'HUMS':'FHUMS'}

replace_sch = {'None - Faculty Office':'Faculty Office'}

replace_d = {'Technical Services':'Technical'}

replace_in = {'Henry Royce Institute for Advanced Materials (HRI)':'Henry Royce Institute',
             'Photon Science Institute (PSI)':'Photon Science Institute',
              'Manchester Institute of Biotechnology (MIB)':'Manchester Institute of Biotechnology',
             'Manchester Environmental Research Institute (MERI)':'Manchester Environmental Research Institute'}

replacement = [replace_sg,replace_fc,replace_sch,replace_d,replace_in]

for index, ele in enumerate(replacement):
    for old_value, new_value in ele.items():
        sv_stakeholders.iloc[:,index]=sv_stakeholders.iloc[:,index].str.replace(old_value, new_value)

sv_stakeholders.fillna('Unknown', inplace=True)


Merging 

In [776]:
Stakeholder_Data = pd.concat([it_stakeholders,sv_stakeholders],axis=0)
Stakeholder_Data.to_csv('Stakeholder_Data.csv', index = True)
Stakeholder_Data.to_excel('Stakeholder_Data.xlsx',engine='openpyxl', index = True)

Visialization